# Second Layer: Genesis of Non-Gaussianity

In [3]:
import sympy as sp

from symlib.utils import pull_sums_out_front, sum_kronecker_contract, wilds, wild_subs
from symlib.random import ExpVal, RandomIndexedBase
from symlib.gaussian import GaussianIndexedBase, wick_contraction
from symlib.MLP import ZLayer

In [6]:
i1, i2, i3, i4 = sp.symbols('i1 i2 i3 i4', integer=True)
alpha1, alpha2, alpha3, alpha4 = sp.symbols('alpha1, alpha2, alpha3, alpha4', integer=True)

# Input
sigma1 = RandomIndexedBase('sigma^(1)')
n1 = sp.symbols('n1', integer=True)

# Layer param
b = GaussianIndexedBase('b^(2)')
W = GaussianIndexedBase('W^(2)')
C_b, C_w = sp.symbols('C_b^(2) C_W^(2)', positive=True)

In [7]:
z2 = ZLayer(W, b, sigma1, n1)